In [1]:
import yaml

import utils

In [2]:
with open("diag_metadata.yaml", mode="r") as fptr:
    diag_metadata_list = yaml.safe_load(fptr)

varnames = utils.get_varnames_from_metadata_list(diag_metadata_list)

In [3]:
casename = "g.e22.G1850ECO_JRA_HR.TL319_t13.004"

case_hist = utils.CaseClass(casename)
case_ts = utils.CaseClass(casename)

stream = "pop.h"
case_hist._timeseries_filenames[stream] = []

for year in range(1, 62):
    print(f"Starting year {year:04}...")
    all_same = True
    found_ts_and_hist = True
    year_found = True
    for diag_metadata in diag_metadata_list:
        varname = diag_metadata["varname"]

        # (1) generate datasets
        try:
            ds_hist = case_hist.gen_dataset(
                varname, stream, start_year=year, end_year=year, quiet=True
            )
            ds_ts = case_ts.gen_dataset(
                varname, stream, start_year=year, end_year=year, quiet=True
            )
        except ValueError:
            year_found = False
            break

        # (2) Check sources of data being read
        data_source = case_hist.get_dataset_source(stream, year, varname)
        if data_source != "hist":
            print(
                f"case_hist data for {varname} comes from '{data_source}', not history files"
            )
            found_ts_and_hist = False
            break
        data_source = case_ts.get_dataset_source(stream, year, varname)
        if data_source != "time series":
            print(
                f"case_ts data for {varname} comes from '{data_source}', not time series files"
            )
            found_ts_and_hist = False
            break

        # (3) Compare datasets
        da_hist = ds_hist[varname].isel(diag_metadata.get("isel_dict"))
        da_ts = ds_ts[varname].isel(diag_metadata.get("isel_dict"))
        if not da_hist.identical(da_ts):
            print(f"{varname} is different in year {year:04}")
            all_same = False

    # Error checking after running through all variables for a given year
    # (1) was data for the year available?
    if not year_found:
        print(f"Year {year:04} is not available, are you sure it has been run?")
        break

    # (2) was data for the year available via both time series and history files?
    if not found_ts_and_hist:
        print(
            f"Either time series or history files for variables in {year:04} are missing, skipping comparison"
        )
        print("----")
        continue

    # (3) was the data in the time series files identical to that in the history files?
    if all_same:
        print(f"No differences found in year {year:04}")
    print(f"Finished {year:04}")
    print("----")

Starting year 0001...
case_ts data for POC_FLUX_100m comes from 'hist', not time series files
Either time series or history files for variables in 0001 are missing, skipping comparison
----
Starting year 0002...
case_ts data for POC_FLUX_100m comes from 'hist', not time series files
Either time series or history files for variables in 0002 are missing, skipping comparison
----
Starting year 0003...
case_ts data for POC_FLUX_100m comes from 'hist', not time series files
Either time series or history files for variables in 0003 are missing, skipping comparison
----
Starting year 0004...
case_ts data for POC_FLUX_100m comes from 'hist', not time series files
Either time series or history files for variables in 0004 are missing, skipping comparison
----
Starting year 0005...
case_ts data for POC_FLUX_100m comes from 'hist', not time series files
Either time series or history files for variables in 0005 are missing, skipping comparison
----
Starting year 0006...
case_ts data for POC_FLUX_10